# Medyczne zastosowania uczenia maszynowego - zadanie 2a

# Temat: Selekcja istotnych cech - XGBoost

Analiza zbiorów danych dotyczących raka prostaty, zawierających próbki moczu, surowicy i tkanek

Katarzyna Pieczonka, nr indeksu 132785 <br/>
Informatyka II stopień, semestr zimowy <br/>
rok akademicki 2024/25

1. Wczytanie bibliotek i zbiorów danych

In [49]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


#Odczytanie zbiorów danych
datasetM = pd.read_csv('FTIR_800-1800_mocz_1chory_0zdrowy.csv', delimiter=';', dtype=float, decimal=',') #Dane rozdzielone są średnikami
datasetS = pd.read_csv('FTIR_800-1800_surowica_1chory_0zdrowy.csv', delimiter=';', dtype=float, decimal=',')
datasetT = pd.read_csv('FTIR_800-1800_tkanka_1chory_0zdrowy.csv', delimiter=';', dtype=float, decimal=',')

noColumn1 = datasetM.shape[1]
noColumn2 = datasetS.shape[1]
noColumn3 = datasetT.shape[1]

print(f"Dataset1 - próbki moczu: \n\n {datasetM.head()}")
print(f"Dataset2 - próbki surowicy: \n\n {datasetS.head()}")
print(f"Dataset3 - próbki tkanek: \n\n {datasetT.head()}")

column_namesM = datasetM.columns[:-1] #Wszystkie nazwy kolumn w każdym ze zbiorów danych
column_namesS = datasetS.columns[:-1]
column_namesT = datasetT.columns[:-1]

Dataset1 - próbki moczu: 

    800.31372  800.79584  801.27796  801.76007  802.24219  802.72431  \
0    0.15608    0.15398    0.15202    0.15021    0.14849    0.14676   
1    0.18841    0.18582    0.18343    0.18128    0.17932    0.17746   
2    0.21911    0.21687    0.21493    0.21324    0.21162    0.20987   
3    0.12812    0.12601    0.12409    0.12236    0.12073    0.11910   
4    0.16763    0.16522    0.16306    0.16113    0.15932    0.15754   

   803.20642  803.68854  804.17066  804.65277  ...  1796.84919  1797.3313  \
0    0.14498    0.14316    0.14136    0.13965  ...     0.02749    0.02584   
1    0.17564    0.17384    0.17212    0.17054  ...     0.02380    0.02214   
2    0.20785    0.20556    0.20316    0.20084  ...     0.02588    0.02379   
3    0.11740    0.11562    0.11382    0.11209  ...     0.02655    0.02465   
4    0.15569    0.15379    0.15190    0.15012  ...     0.02583    0.02380   

   1797.81342  1798.29554  1798.77765  1799.25977  1799.74189  1800.224  \
0     0

2. Przygotowanie zbiorów danych - sprawdzanie brakujących wartości i usuwanie duplikatów.

In [50]:
#Sprawdzanie braków danych w dataset1
print("Liczba brakujących wartości dla poszczególnych kolumn w dataset1:")
missing_values = datasetM.isnull().sum()
for col, value in missing_values.items():
    if value != 0:
        print(f"{col}, brak: {value} wartości")

#Sprawdzanie duplikatów w dataset1
duplicates = datasetM.duplicated().sum()
print(f"Duplikaty: {duplicates}")
print("Usuwanie duplikatów: ")
datasetM = datasetM.drop_duplicates() #Funkcja usuwająca duplikaty
duplicates = datasetM.duplicated().sum()
print(f"Duplikaty: {duplicates}")

#####################################################

#Sprawdzanie braków danych w dataset2
print("Liczba brakujących wartości dla poszczególnych kolumn w dataset2:")
missing_values2 = datasetS.isnull().sum()
for col, value in missing_values2.items():
    if value != 0:
        print(f"{col}, brak: {value} wartości")

#Sprawdzanie duplikatów w dataset2
duplicates2 = datasetS.duplicated().sum()
print(f"Duplikaty: {duplicates2}")
print("Usuwanie duplikatów: ")
datasetS = datasetS.drop_duplicates()
duplicates2 = datasetS.duplicated().sum()
print(f"Duplikaty: {duplicates2}")

####################################################

#Sprawdzanie braków danych w dataset3
print("Liczba brakujących wartości dla poszczególnych kolumn w dataset3:")
missing_values3 = datasetT.isnull().sum()
for col, value in missing_values3.items():
    if value != 0:
        print(f"{col}, brak: {value} wartości")

#Sprawdzanie duplikatów w dataset3
duplicates3 = datasetT.duplicated().sum()
print(f"Duplikaty: {duplicates3}")
print("Usuwanie duplikatów: ")
datasetT = datasetT.drop_duplicates()
duplicates3 = datasetT.duplicated().sum()
print(f"Duplikaty: {duplicates3}")

Liczba brakujących wartości dla poszczególnych kolumn w dataset1:
Duplikaty: 2
Usuwanie duplikatów: 
Duplikaty: 0
Liczba brakujących wartości dla poszczególnych kolumn w dataset2:
Duplikaty: 0
Usuwanie duplikatów: 
Duplikaty: 0
Liczba brakujących wartości dla poszczególnych kolumn w dataset3:
Duplikaty: 2
Usuwanie duplikatów: 
Duplikaty: 0


# Zbiór1 - próbki moczu

3. Podział na zbiór treningowy (70%), testowy (10%) i walidacyjny (20%)

In [51]:
#Podział cech i etykiet
X_mocz = datasetM.iloc[:,:-1].values #cechy
y_mocz = datasetM.iloc[:,-1].values #etykiety

#Podział na zbiór treningowy 70%, testowy 30% (parametr test_size) oraz pozostałą część
X_train, X_temp, y_train, y_temp = train_test_split(X_mocz, y_mocz, test_size=0.3, random_state=42)

#Podział pozostałej części na zbiór testowy i walidacyjny
X_val, X_test, y_val, y_test = train_test_split(X_mocz, y_mocz, test_size=0.33, random_state=42)

2. W każdym podziale na zbiorze treningowym i z użyciem zbioru walidacyjnego (hiperparametry) wykonujemy selekcję cech wykorzystując algorytm XGBoost (10%, 25%, 50% najbardziej istotnych cech stosując miary istotności “gain”, “weight”, “cover”, “total_gain”, “total_cover”). 

In [52]:
#Tworzymy i trenujemy model XGBoost
model = xgb.XGBClassifier(eval_metric='logloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

Obliczanie ważności cech dla miary 'gain':

In [53]:
feature_importances = model.get_booster().get_score(importance_type='gain')
feature_importances = pd.DataFrame({
    'Feature': [column_namesM[int(f[1:])] for f in feature_importances.keys()],
    'Gain': feature_importances.values()
})

#w Feature jest specjalne dopasowanie do oryginalnych nazw kolumn, zamiast nowych nazw nadawanych przez XGBoost

# Sortowanie cech według istotności wg miary Gain
feature_importances = feature_importances.sort_values(by='Gain', ascending=False) #ascending = False, czyli cechy sortowane są malejąco

# Liczba cech do zachowania w każdym przypadku
n_features_10_percent = int(0.1 * len(feature_importances))
n_features_25_percent = int(0.25 * len(feature_importances))
n_features_50_percent = int(0.5 * len(feature_importances))

# Selekcja 10%, 25%, 50% najbardziej istotnych cech według miary Gain
selected_features_10_gain = feature_importances['Feature'].values[:n_features_10_percent]
selected_features_25_gain = feature_importances['Feature'].values[:n_features_25_percent]
selected_features_50_gain = feature_importances['Feature'].values[:n_features_50_percent]

Dla miary 'weight':

In [54]:
feature_importances = model.get_booster().get_score(importance_type='weight')
feature_importances = pd.DataFrame({
    'Feature': [column_namesM[int(f[1:])] for f in feature_importances.keys()],
    'Weight': feature_importances.values()
})

# Sortowanie cech według istotności
feature_importances = feature_importances.sort_values(by='Weight', ascending=False)

# Liczba cech do zachowania w każdym przypadku
n_features_10_percent = int(0.1 * len(feature_importances))
n_features_25_percent = int(0.25 * len(feature_importances))
n_features_50_percent = int(0.5 * len(feature_importances))

# Selekcja 10%, 25%, 50% najbardziej istotnych cech
selected_features_10_weight = feature_importances['Feature'].values[:n_features_10_percent]
selected_features_25_weight = feature_importances['Feature'].values[:n_features_25_percent]
selected_features_50_weight = feature_importances['Feature'].values[:n_features_50_percent]

Dla miary 'cover':

In [55]:
feature_importances = model.get_booster().get_score(importance_type='cover')
feature_importances = pd.DataFrame({
    'Feature': [column_namesM[int(f[1:])] for f in feature_importances.keys()],
    'Cover': feature_importances.values()
})

# Sortowanie cech według istotności
feature_importances = feature_importances.sort_values(by='Cover', ascending=False)

# Liczba cech do zachowania w każdym przypadku
n_features_10_percent = int(0.1 * len(feature_importances))
n_features_25_percent = int(0.25 * len(feature_importances))
n_features_50_percent = int(0.5 * len(feature_importances))

# Selekcja 10%, 25%, 50% najbardziej istotnych cech
selected_features_10_cover = feature_importances['Feature'].values[:n_features_10_percent]
selected_features_25_cover = feature_importances['Feature'].values[:n_features_25_percent]
selected_features_50_cover = feature_importances['Feature'].values[:n_features_50_percent]

Dla miary 'total_gain':

In [56]:
feature_importances = model.get_booster().get_score(importance_type='total_gain')
feature_importances = pd.DataFrame({
    'Feature': [column_namesM[int(f[1:])] for f in feature_importances.keys()],
    'Total_Gain': feature_importances.values()
})

# Sortowanie cech według istotności
feature_importances = feature_importances.sort_values(by='Total_Gain', ascending=False)

# Liczba cech do zachowania w każdym przypadku
n_features_10_percent = int(0.1 * len(feature_importances))
n_features_25_percent = int(0.25 * len(feature_importances))
n_features_50_percent = int(0.5 * len(feature_importances))

# Selekcja 10%, 25%, 50% najbardziej istotnych cech
selected_features_10_total_gain = feature_importances['Feature'].values[:n_features_10_percent]
selected_features_25_total_gain = feature_importances['Feature'].values[:n_features_25_percent]
selected_features_50_total_gain = feature_importances['Feature'].values[:n_features_50_percent]

Dla miary 'total_cover':

In [57]:
feature_importances = model.get_booster().get_score(importance_type='total_cover')
feature_importances = pd.DataFrame({
    'Feature': [column_namesM[int(f[1:])] for f in feature_importances.keys()],
    'Total_cover': feature_importances.values()
})

# Sortowanie cech według istotności
feature_importances = feature_importances.sort_values(by='Total_cover', ascending=False)

# Liczba cech do zachowania w każdym przypadku
n_features_10_percent = int(0.1 * len(feature_importances))
n_features_25_percent = int(0.25 * len(feature_importances))
n_features_50_percent = int(0.5 * len(feature_importances))

# Selekcja 10%, 25%, 50% najbardziej istotnych cech
selected_features_10_total_cover = feature_importances['Feature'].values[:n_features_10_percent]
selected_features_25_total_cover = feature_importances['Feature'].values[:n_features_25_percent]
selected_features_50_total_cover = feature_importances['Feature'].values[:n_features_50_percent]

W efekcie możemy wyświetlić wybrane cechy według wszystkich powyższych miar istotności

In [58]:
print("10% najważniejszych cech według Gain:", selected_features_10_gain)
print("10% najważniejszych cech według Weight:", selected_features_10_weight)
print("10% najważniejszych cech według Cover:", selected_features_10_cover)
print("10% najważniejszych cech według Total Gain:", selected_features_10_total_gain)
print("10% najważniejszych cech według Total Cover:", selected_features_10_total_cover)

print("25% najważniejszych cech według Gain:", selected_features_25_gain)
print("25% najważniejszych cech według Weight:", selected_features_25_weight)
print("25% najważniejszych cech według Cover:", selected_features_25_cover)
print("25% najważniejszych cech według Total Gain:", selected_features_25_total_gain)
print("25% najważniejszych cech według Total Cover:", selected_features_25_total_cover)

print("50% najważniejszych cech według Gain:", selected_features_50_gain)
print("50% najważniejszych cech według Weight:", selected_features_50_weight)
print("50% najważniejszych cech według Cover:", selected_features_50_cover)
print("50% najważniejszych cech według Total Gain:", selected_features_50_total_gain)
print("50% najważniejszych cech według Total Cover:", selected_features_50_total_cover)

10% najważniejszych cech według Gain: ['1613.16268']
10% najważniejszych cech według Weight: ['1615.57326']
10% najważniejszych cech według Cover: ['883.23782']
10% najważniejszych cech według Total Gain: ['1615.57326']
10% najważniejszych cech według Total Cover: ['1615.57326']
25% najważniejszych cech według Gain: ['1613.16268' '1615.57326' '1611.71633']
25% najważniejszych cech według Weight: ['1615.57326' '1611.71633' '1613.16268']
25% najważniejszych cech według Cover: ['883.23782' '896.25497' '1615.57326']
25% najważniejszych cech według Total Gain: ['1615.57326' '1613.16268' '1611.71633']
25% najważniejszych cech według Total Cover: ['1615.57326' '1613.16268' '1611.71633']
50% najważniejszych cech według Gain: ['1613.16268' '1615.57326' '1611.71633' '1612.19845' '883.23782'
 '1210.11302' '896.25497']
50% najważniejszych cech według Weight: ['1615.57326' '1611.71633' '1613.16268' '1612.19845' '883.23782'
 '896.25497' '1210.11302']
50% najważniejszych cech według Cover: ['883.2378

In [59]:
from sklearn.model_selection import GridSearchCV

# Przekształcenie listy wybranych nazw najlepszych cech na indeksy
selected_indices_10_gain = [column_namesM.get_loc(feature) for feature in selected_features_10_gain]

# Przykładowe wybranie 10% cech według Gain
X_train_selected = X_train[:, selected_indices_10_gain]
X_test_selected = X_test[:, selected_indices_10_gain]
X_val_selected = X_val[:, selected_indices_10_gain]

# Definicja hiperparametrów do optymalizacji
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300]
}

# Optymalizacja hiperparametrów z użyciem zbioru walidacyjnego
grid_search = GridSearchCV(xgb.XGBClassifier(eval_metric='logloss'),
                           param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_selected, y_train)
#Przy użyciu GridSearchCV model trenowany jest na różnych kombinacjach wcześniej zdefiniowanych hiperparametrów

# Wyniki optymalizacji, czyli najlepsze parametry, dla których osiągana jest najwyższa dokładność
print("Najlepsze parametry:", grid_search.best_params_)
print("Najlepsza dokładność:", grid_search.best_score_)


Najlepsze parametry: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Najlepsza dokładność: 0.9740384615384615


In [60]:
selected_indices_25_cover = [column_namesM.get_loc(feature) for feature in selected_features_25_cover]

# Przykładowe wybranie 25% cech według Cover
X_train_selected_cover = X_train[:, selected_indices_25_cover]
X_test_selected_cover = X_test[:, selected_indices_25_cover]
X_val_selected_cover = X_val[:, selected_indices_25_cover]

# Definicja hiperparametrów do optymalizacji
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2], #współczynnik uczenia
    'max_depth': [3, 5, 7],  #maksymalna głębokość drzewa
    'n_estimators': [100, 200, 300] #liczba drzew w modelu
}

# Optymalizacja hiperparametrów z użyciem zbioru walidacyjnego
grid_search = GridSearchCV(xgb.XGBClassifier(eval_metric='logloss'),
                           param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_selected, y_train)

#eval_metric='logloss' - model minimalizuje logarytmiczną stratę
#cv=3 - liczba podziałów walidacji krzyżowej

# Wyniki optymalizacji
print("Najlepsze parametry:", grid_search.best_params_)
print("Najlepsza dokładność:", grid_search.best_score_)

Najlepsze parametry: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Najlepsza dokładność: 0.9740384615384615


3. Klasyfikujemy zbiór testowy modelem zbudowanym na zbiorze treningowym i zbiorze selekcjonowanych cech. Zastosować dowolny klasyfikator, np. kNN, SVM lub C5.0

In [61]:
#Wybrany klasyfikator: SVM
#Tworzenie i trenowanie modelu SVM na zbiorze treningowym z wybranymi cechami - 10% najlepszych cech
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_selected, y_train)

# Przewidywanie na zbiorze testowym
y_pred = svm_model.predict(X_test_selected)

# Ocena modelu
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność podczas treningu na zb. z wybranymi cechami - 10%: {accuracy:.4f}")

# Szczegółowy raport z wyników
print("\nRaport klasyfikacji:")
print(classification_report(y_test, y_pred))

#Trenowanie modelu na zbiorze treningowym z wybranymi cechami - 50% najlepszych cech
selected_indices_25_gain = [column_namesM.get_loc(feature) for feature in selected_features_25_gain]
X_train_selected25 = X_train[:, selected_indices_25_gain]
X_test_selected25 = X_test[:, selected_indices_25_gain]
svm_model.fit(X_train_selected25, y_train)
y_pred = svm_model.predict(X_test_selected25)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność podczas treningu na zbiorze z 25% cech: {accuracy:.4f}")
print("\nRaport klasyfikacji:")
print(classification_report(y_test, y_pred))

#Trenowanie modelu na zbiorze treningowym z wybranymi cechami - 50% najlepszych cech
selected_indices_50_gain = [column_namesM.get_loc(feature) for feature in selected_features_50_gain]
X_train_selected50 = X_train[:, selected_indices_50_gain]
X_test_selected50 = X_test[:, selected_indices_50_gain]
svm_model.fit(X_train_selected50, y_train)
y_pred = svm_model.predict(X_test_selected50)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność podczas treningu na zbiorze z 50% cech: {accuracy:.4f}")
print("\nRaport klasyfikacji:")
print(classification_report(y_test, y_pred))

#Trenowanie modelu na całym zbiorze treningowym
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność podczas treningu na całym zbiorze: {accuracy:.4f}")
print("\nRaport klasyfikacji:")
print(classification_report(y_test, y_pred))

Dokładność podczas treningu na zb. z wybranymi cechami - 10%: 0.9239

Raport klasyfikacji:
              precision    recall  f1-score   support

         0.0       1.00      0.83      0.91        42
         1.0       0.88      1.00      0.93        50

    accuracy                           0.92        92
   macro avg       0.94      0.92      0.92        92
weighted avg       0.93      0.92      0.92        92

Dokładność podczas treningu na zbiorze z 25% cech: 1.0000

Raport klasyfikacji:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        42
         1.0       1.00      1.00      1.00        50

    accuracy                           1.00        92
   macro avg       1.00      1.00      1.00        92
weighted avg       1.00      1.00      1.00        92

Dokładność podczas treningu na zbiorze z 50% cech: 1.0000

Raport klasyfikacji:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1

4. Porównanie wyników dla pełnego zbioru cech i zbioru selekcjonowanego: Na podstawie powyższych wyników można stwierdzić, że:
- najlepsze hiperparametry dla modelu, wspólne dla wszystkich testowanych zestawów cech, to learning_rate = 0.1, max_depth = 3, n_estimators = 100
- model osiągnął dokładność ~92% dla 10% cech, ale już dla 25% cech i więcej dokładność wynosiła 100%. 10% cech jest więc niewystarczające, aby model nauczył się wszystkich wzorców w danych, wystarczy do tego natomiast 25% cech;
- w przypadku 10% cech, model bardzo dobrze rozpoznaje klasę 1, ale w przypadku klasy 0 (czyli zdrowy), czułość (recall) wynosi tylko 0.83, co sugeruje, że model nie rozpoznaje dobrze wszystkich zdrowych przypadków, myląc je z chorymi. Jeśli model jest trenowany na 25% i więcej cech, ten problem nie występuje - obie klasy rozpoznawane są prawidłowo.

# Zbiór2 - próbki surowicy

3. Podział na zbiór treningowy (70%), testowy (10%) i walidacyjny (20%)

In [62]:
#Podział cech i etykiet
X_surowica = datasetS.iloc[:,:-1].values 
y_surowica = datasetS.iloc[:,-1].values

#Podział na zbiór treningowy 70%, testowy 30% (parametr test_size) oraz pozostałą część
X_train, X_temp, y_train, y_temp = train_test_split(X_surowica, y_surowica, test_size=0.3, random_state=42)

#Podział pozostałej części na zbiór testowy i walidacyjny
X_val, X_test, y_val, y_test = train_test_split(X_surowica, y_surowica, test_size=0.33, random_state=42)

2. W każdym podziale na zbiorze treningowym i z użyciem zbioru walidacyjnego (hiperparametry) wykonujemy selekcję cech wykorzystując algorytm XGBoost (10%, 25%, 50% najbardziej istotnych cech stosując miary istotności “gain”, “weight”, “cover”, “total_gain”, “total_cover”). 

In [63]:
#Tworzymy i trenujemy model XGBoost
model = xgb.XGBClassifier(eval_metric='logloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

Obliczanie ważności cech dla wszystkich miar po kolei:

In [64]:
importance_types = ['gain','weight', 'cover', 'total_gain', 'total_cover']
selected_features = {}

for importance_type in importance_types:
    feature_importances = model.get_booster().get_score(importance_type=importance_type)
    feature_importances = pd.DataFrame({
        'Feature': [column_namesS[int(f[1:])] for f in feature_importances.keys()],
        importance_type.capitalize(): feature_importances.values()
    }).sort_values(by=importance_type.capitalize(), ascending=False)

    # Wybór 10%, 25%, 50% najbardziej istotnych cech
    selected_features[f'{importance_type}_10'] = feature_importances['Feature'].values[:n_features_10_percent]
    selected_features[f'{importance_type}_25'] = feature_importances['Feature'].values[:n_features_25_percent]
    selected_features[f'{importance_type}_50'] = feature_importances['Feature'].values[:n_features_50_percent]

print("10% najważniejszych cech według Gain:", selected_features['gain_10'])
print("10% najważniejszych cech według Weight:", selected_features['weight_10'])
print("10% najważniejszych cech według Cover:", selected_features['cover_10'])
print("10% najważniejszych cech według Total Gain:", selected_features['total_gain_10'])
print("10% najważniejszych cech według Total Cover:", selected_features['total_cover_10'])

print("25% najważniejszych cech według Gain:", selected_features['gain_25'])
print("25% najważniejszych cech według Weight:", selected_features['weight_25'])
print("25% najważniejszych cech według Cover:", selected_features['cover_25'])
print("25% najważniejszych cech według Total Gain:", selected_features['total_gain_25'])
print("25% najważniejszych cech według Total Cover:", selected_features['total_cover_25'])

print("50% najważniejszych cech według Gain:", selected_features['gain_50'])
print("50% najważniejszych cech według Weight:", selected_features['weight_50'])
print("50% najważniejszych cech według Cover:", selected_features['cover_50'])
print("50% najważniejszych cech według Total Gain:", selected_features['total_gain_50'])
print("50% najważniejszych cech według Total Cover:", selected_features['total_cover_50'])

10% najważniejszych cech według Gain: ['1662.82071']
10% najważniejszych cech według Weight: ['1636.30429']
10% najważniejszych cech według Cover: ['1792.99225']
10% najważniejszych cech według Total Gain: ['1635.82217']
10% najważniejszych cech według Total Cover: ['1636.30429']
25% najważniejszych cech według Gain: ['1662.82071' '1668.124' '1631.96523']
25% najważniejszych cech według Weight: ['1636.30429' '1635.82217' '1507.5791']
25% najważniejszych cech według Cover: ['1792.99225' '1631.96523' '1662.82071']
25% najważniejszych cech według Total Gain: ['1635.82217' '1636.30429' '1662.82071']
25% najważniejszych cech według Total Cover: ['1636.30429' '1635.82217' '1779.9751']
50% najważniejszych cech według Gain: ['1662.82071' '1668.124' '1631.96523' '1792.99225' '1653.17837'
 '1786.24262' '1632.44735']
50% najważniejszych cech według Weight: ['1636.30429' '1635.82217' '1507.5791' '1779.9751' '1688.85502'
 '1628.59042' '815.74146']
50% najważniejszych cech według Cover: ['1792.99225

In [65]:
# Przekształcenie listy wybranych nazw najlepszych cech na indeksy
selected_indices_50_weight = [column_namesS.get_loc(feature) for feature in selected_features['weight_50']]

# Przykładowe wybranie 10% cech według Gain
X_train_selectedS = X_train[:, selected_indices_50_weight]
X_test_selectedS = X_test[:, selected_indices_50_weight]
X_val_selectedS = X_val[:, selected_indices_50_weight]

# Definicja hiperparametrów do optymalizacji
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300]
}

# Optymalizacja hiperparametrów z użyciem zbioru walidacyjnego
grid_search = GridSearchCV(xgb.XGBClassifier(eval_metric='logloss'),
                           param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_selectedS, y_train)

# Wyniki optymalizacji
print("Najlepsze parametry:", grid_search.best_params_)
print("Najlepsza dokładność:", grid_search.best_score_)

Najlepsze parametry: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300}
Najlepsza dokładność: 0.907051282051282


3. Klasyfikujemy zbiór testowy modelem zbudowanym na zbiorze treningowym i zbiorze selekcjonowanych cech. Zastosować dowolny klasyfikator, np. kNN, SVM lub C5.0

In [66]:
#Wybrany klasyfikator: SVM
#Tworzenie i trenowanie modelu SVM na zbiorze treningowym z wybranymi cechami - 10% najlepszych cech
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_selectedS, y_train)

# Przewidywanie na zbiorze testowym
y_pred = svm_model.predict(X_test_selectedS)

# Ocena modelu
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność podczas treningu na zb. z wybranymi cechami - 10%: {accuracy:.4f}")

# Szczegółowy raport z wyników
print("\nRaport klasyfikacji:")
print(classification_report(y_test, y_pred, zero_division=1))
#zero_division = 1 ustawia precyzję na 1 dla klas, które nigdy nie zostały przypisane
#może się tak zdarzyć, gdy model jest zbyt mocno zoptymalizowany na wykrywanie jednej klasy kosztem innej
#lub gdy model nie przypisuje żadnej próbki do jednej z klas

#Trenowanie modelu na zbiorze treningowym z wybranymi cechami - 50% najlepszych cech
selected_indices_25_gain = [column_namesS.get_loc(feature) for feature in selected_features_25_gain]
X_train_selected25 = X_train[:, selected_indices_25_gain]
X_test_selected25 = X_test[:, selected_indices_25_gain]
svm_model.fit(X_train_selected25, y_train)
y_pred = svm_model.predict(X_test_selected25)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność podczas treningu na zbiorze z 25% cech: {accuracy:.4f}")
print("\nRaport klasyfikacji:")
print(classification_report(y_test, y_pred, zero_division=1))

#Trenowanie modelu na zbiorze treningowym z wybranymi cechami - 50% najlepszych cech
selected_indices_50_gain = [column_namesS.get_loc(feature) for feature in selected_features_50_gain]
X_train_selected50 = X_train[:, selected_indices_50_gain]
X_test_selected50 = X_test[:, selected_indices_50_gain]
svm_model.fit(X_train_selected50, y_train)
y_pred = svm_model.predict(X_test_selected50)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność podczas treningu na zbiorze z 50% cech: {accuracy:.4f}")
print("\nRaport klasyfikacji:")
print(classification_report(y_test, y_pred, zero_division=1))

#Trenowanie modelu na całym zbiorze treningowym
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność podczas treningu na całym zbiorze: {accuracy:.4f}")
print("\nRaport klasyfikacji:")
print(classification_report(y_test, y_pred, zero_division=1))

Dokładność podczas treningu na zb. z wybranymi cechami - 10%: 0.7065

Raport klasyfikacji:
              precision    recall  f1-score   support

         0.0       0.73      0.57      0.64        42
         1.0       0.69      0.82      0.75        50

    accuracy                           0.71        92
   macro avg       0.71      0.70      0.70        92
weighted avg       0.71      0.71      0.70        92

Dokładność podczas treningu na zbiorze z 25% cech: 0.5435

Raport klasyfikacji:
              precision    recall  f1-score   support

         0.0       1.00      0.00      0.00        42
         1.0       0.54      1.00      0.70        50

    accuracy                           0.54        92
   macro avg       0.77      0.50      0.35        92
weighted avg       0.75      0.54      0.38        92

Dokładność podczas treningu na zbiorze z 50% cech: 0.5435

Raport klasyfikacji:
              precision    recall  f1-score   support

         0.0       1.00      0.00      0

4. Porównanie wyników dla pełnego zbioru cech i zbioru selekcjonowanego: Na podstawie powyższych wyników można stwierdzić, że:
- najlepsze hiperparametry dla modelu, wspólne dla wszystkich testowanych zestawów cech, to learning_rate = 0.1, max_depth = 3, n_estimators = 300
- model osiągnął dokładność ~71% dla 10% cech, natomiast dla 25% cech i 50% cech ta dokładność spadała. Model całkowicie pomijał jedną klasę (0 - zdrowy, precyzja wynosiła 0.0). Świadczy to o przeuczeniu modelu lub niewystarczającej reprezentacji tej klasy w danych, co uniemożliwiało modelowi wychwycenie wzorca. Dla całego zbioru danych dokładność była niewiele wyższa niż w przypadku 10% cech. Może to świadczyć, że zbiór ten jest niewystarczający, by model nauczył się poprawnie rozpoznawać obie klasy
- dla całego zbioru danych, model osiągał bardziej zrównoważone wyniki dla obu klas. Cały zbiór danych posiada więc kluczowe informacje, które umożliwiają lepszą klasyfikację niż podczas wybrania konkretnych cech.

# Zbiór3 - próbki tkanek

3. Podział na zbiór treningowy (70%), testowy (10%) i walidacyjny (20%)

In [67]:
#Podział cech i etykiet
X_tkanka = datasetT.iloc[:,:-1].values 
y_tkanka = datasetT.iloc[:,-1].values

#Podział na zbiór treningowy 70%, testowy 30% (parametr test_size) oraz pozostałą część
X_train, X_temp, y_train, y_temp = train_test_split(X_tkanka, y_tkanka, test_size=0.3, random_state=42)

#Podział pozostałej części na zbiór testowy i walidacyjny
X_val, X_test, y_val, y_test = train_test_split(X_tkanka, y_tkanka, test_size=0.33, random_state=42)

2. W każdym podziale na zbiorze treningowym i z użyciem zbioru walidacyjnego (hiperparametry) wykonujemy selekcję cech wykorzystując algorytm XGBoost (10%, 25%, 50% najbardziej istotnych cech stosując miary istotności “gain”, “weight”, “cover”, “total_gain”, “total_cover”). 

In [68]:
#Tworzymy i trenujemy model XGBoost
model = xgb.XGBClassifier(eval_metric='logloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

Obliczanie ważności cech dla wszystkich miar po kolei:

In [69]:
importance_types = ['gain','weight', 'cover', 'total_gain', 'total_cover']
selected_features = {}

for importance_type in importance_types:
    feature_importances = model.get_booster().get_score(importance_type=importance_type)
    feature_importances = pd.DataFrame({
        'Feature': [column_namesT[int(f[1:])] for f in feature_importances.keys()],
        importance_type.capitalize(): feature_importances.values()
    }).sort_values(by=importance_type.capitalize(), ascending=False)

    # Wybór 10%, 25%, 50% najbardziej istotnych cech
    selected_features[f'{importance_type}_10'] = feature_importances['Feature'].values[:n_features_10_percent]
    selected_features[f'{importance_type}_25'] = feature_importances['Feature'].values[:n_features_25_percent]
    selected_features[f'{importance_type}_50'] = feature_importances['Feature'].values[:n_features_50_percent]

print("10% najważniejszych cech według Gain:", selected_features['gain_10'])
print("10% najważniejszych cech według Weight:", selected_features['weight_10'])
print("10% najważniejszych cech według Cover:", selected_features['cover_10'])
print("10% najważniejszych cech według Total Gain:", selected_features['total_gain_10'])
print("10% najważniejszych cech według Total Cover:", selected_features['total_cover_10'])

print("25% najważniejszych cech według Gain:", selected_features['gain_25'])
print("25% najważniejszych cech według Weight:", selected_features['weight_25'])
print("25% najważniejszych cech według Cover:", selected_features['cover_25'])
print("25% najważniejszych cech według Total Gain:", selected_features['total_gain_25'])
print("25% najważniejszych cech według Total Cover:", selected_features['total_cover_25'])

print("50% najważniejszych cech według Gain:", selected_features['gain_50'])
print("50% najważniejszych cech według Weight:", selected_features['weight_50'])
print("50% najważniejszych cech według Cover:", selected_features['cover_50'])
print("50% najważniejszych cech według Total Gain:", selected_features['total_gain_50'])
print("50% najważniejszych cech według Total Cover:", selected_features['total_cover_50'])

10% najważniejszych cech według Gain: ['907.82577']
10% najważniejszych cech według Weight: ['907.82577']
10% najważniejszych cech według Cover: ['907.82577']
10% najważniejszych cech według Total Gain: ['907.82577']
10% najważniejszych cech według Total Cover: ['907.82577']
25% najważniejszych cech według Gain: ['907.82577']
25% najważniejszych cech według Weight: ['907.82577']
25% najważniejszych cech według Cover: ['907.82577']
25% najważniejszych cech według Total Gain: ['907.82577']
25% najważniejszych cech według Total Cover: ['907.82577']
50% najważniejszych cech według Gain: ['907.82577']
50% najważniejszych cech według Weight: ['907.82577']
50% najważniejszych cech według Cover: ['907.82577']
50% najważniejszych cech według Total Gain: ['907.82577']
50% najważniejszych cech według Total Cover: ['907.82577']


In [70]:
# Przekształcenie listy wybranych nazw najlepszych cech na indeksy
selected_indices_50_weight = [column_namesT.get_loc(feature) for feature in selected_features['weight_50']]

# Przykładowe wybranie 10% cech według Gain
X_train_selectedT = X_train[:, selected_indices_50_weight]
X_test_selectedT = X_test[:, selected_indices_50_weight]
X_val_selectedT = X_val[:, selected_indices_50_weight]

# Definicja hiperparametrów do optymalizacji
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300]
}

# Optymalizacja hiperparametrów z użyciem zbioru walidacyjnego
grid_search = GridSearchCV(xgb.XGBClassifier(eval_metric='logloss'),
                           param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_selectedT, y_train)

# Wyniki optymalizacji
print("Najlepsze parametry:", grid_search.best_params_)
print("Najlepsza dokładność:", grid_search.best_score_)

Najlepsze parametry: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Najlepsza dokładność: 0.9954337899543378


3. Klasyfikujemy zbiór testowy modelem zbudowanym na zbiorze treningowym i zbiorze selekcjonowanych cech. Zastosować dowolny klasyfikator, np. kNN, SVM lub C5.0

In [71]:
#Wybrany klasyfikator: SVM
#Tworzenie i trenowanie modelu SVM na zbiorze treningowym z wybranymi cechami - 10% najlepszych cech
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_selectedT, y_train)

# Przewidywanie na zbiorze testowym
y_pred = svm_model.predict(X_test_selectedT)

# Ocena modelu
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność podczas treningu na zb. z wybranymi cechami - 10%: {accuracy:.4f}")

# Szczegółowy raport z wyników
print("\nRaport klasyfikacji:")
print(classification_report(y_test, y_pred, zero_division=1))

#Trenowanie modelu na zbiorze treningowym z wybranymi cechami - 50% najlepszych cech
selected_indices_25_gain = [column_namesT.get_loc(feature) for feature in selected_features_25_gain]
X_train_selected25 = X_train[:, selected_indices_25_gain]
X_test_selected25 = X_test[:, selected_indices_25_gain]
svm_model.fit(X_train_selected25, y_train)
y_pred = svm_model.predict(X_test_selected25)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność podczas treningu na zbiorze z 25% cech: {accuracy:.4f}")
print("\nRaport klasyfikacji:")
print(classification_report(y_test, y_pred, zero_division=1))

#Trenowanie modelu na zbiorze treningowym z wybranymi cechami - 50% najlepszych cech
selected_indices_50_gain = [column_namesT.get_loc(feature) for feature in selected_features_50_gain]
X_train_selected50 = X_train[:, selected_indices_50_gain]
X_test_selected50 = X_test[:, selected_indices_50_gain]
svm_model.fit(X_train_selected50, y_train)
y_pred = svm_model.predict(X_test_selected50)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność podczas treningu na zbiorze z 50% cech: {accuracy:.4f}")
print("\nRaport klasyfikacji:")
print(classification_report(y_test, y_pred, zero_division=1))

#Trenowanie modelu na całym zbiorze treningowym
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność podczas treningu na całym zbiorze: {accuracy:.4f}")
print("\nRaport klasyfikacji:")
print(classification_report(y_test, y_pred, zero_division=1))

Dokładność podczas treningu na zb. z wybranymi cechami - 10%: 0.4712

Raport klasyfikacji:
              precision    recall  f1-score   support

         0.0       1.00      0.00      0.00        55
         1.0       0.47      1.00      0.64        49

    accuracy                           0.47       104
   macro avg       0.74      0.50      0.32       104
weighted avg       0.75      0.47      0.30       104

Dokładność podczas treningu na zbiorze z 25% cech: 1.0000

Raport klasyfikacji:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        55
         1.0       1.00      1.00      1.00        49

    accuracy                           1.00       104
   macro avg       1.00      1.00      1.00       104
weighted avg       1.00      1.00      1.00       104

Dokładność podczas treningu na zbiorze z 50% cech: 1.0000

Raport klasyfikacji:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1

4. Porównanie wyników dla pełnego zbioru cech i zbioru selekcjonowanego: Na podstawie powyższych wyników można stwierdzić, że:
- najlepsze hiperparametry dla modelu, wspólne dla wszystkich testowanych zestawów cech, to learning_rate = 0.01, max_depth = 3, n_estimators = 100
- model osiągnął dokładność ~47% dla 10% cech, natomiast dla 25% cech i więcej ta dokładność wynosiła już 100%. Dla 10% cech model zupełnie pomijał klasę 0 - zdrowy. Dla większej ilości cech obie klasy rozpoznawane były już poprawnie. Można więc dojść do wniosku, że 25% cech wystarczy do nauczenia modelu i poprawnej klasyfikacji obu klas